In [1]:
from keras import datasets
import keras
assert keras.backend.image_data_format()  == 'channels_last'

from sklearn import model_selection, metrics
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import os

from keras import backend as K
from keras.utils import np_utils
from keras.models import Model
from keras.layers import *

from keraspp.skeras import *
from keraspp.sfile import *

In [2]:
class CNN(Model):
    def __init__(model, nb_classes, in_shape=None):
        model.nb_classes = nb_classes
        model.in_shape = in_shape
        model.build_model()
        super().__init__(model.x, model.y)
        model.compile()
        
    def build_model(model):
        nb_classes = model.nb_classes
        in_shape = model.in_shape
        
        x = Input(in_shape)
        h = Conv2D(32, kernel_size=(3,3), activation='relu')(x)
        h = Conv2D(64,(3,3),activation='relu')(h)
        h = MaxPooling2D(pool_size=(2,2))(h)
        h = Dropout(0.25)(h)
        h = Flatten()(h)
        z_cl = h
        
        h = Dense(128, activation='relu')(h)
        h = Dropout(0.5)(h)
        z_fl = h
        
        y = Dense(nb_classes, activation='softmax', name='preds')(h)
        
#         model.cl_part = Model(x,z_cl)
#         model.fl_part = Model(x,z_fl)
        
        model.x, model.y = x,y
        
    def compile(model):
        Model.compile(model, loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
        
    

In [3]:
class DataSet:
    def __init__(self,X,y,nb_classes, scaling=True, test_size=0.2, random_state=0):
        self.X = X
        self.add_channels()
        
        X = self.X
        X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=0.2,random_state=random_state)
        
        print(X_train.shape,y_train.shape)
        
        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        
        if scaling:
            scaler = MinMaxScaler()
            n = X_train.shape[0]
            X_train = scaler.fit_transform(X_train.reshape(n,-1)).reshape(X_train.shape)
            n = X_test.shape[0]
            X_test = scaler.fit_transform(X_test.reshape(n,-1)).reshape(X_test.shape)
            self.scaler = scaler
            
            print('X_train shape : ', X_train.shape)
            print(X_train.shape[0], ' train samples')
            print(X_test.shape[0], ' test samples')
            
            Y_train = np_utils.to_categorical(y_train, nb_classes)
            Y_test = np_utils.to_categorical(y_test, nb_classes)
            
            self.X_train, self.X_test = X_train, X_test
            self.Y_train, self.Y_test = Y_train, Y_test
            self.y_train, self.y_test = y_train, y_test
            
    def add_channels(self):
        X = self.X
        
        if len(X.shape) == 3:
            N, img_rows, img_cols = X.shape
            
            if K.image_dim_ordering() == 'th':
                X = X.reshape(X.shape[0],1,img_rows, img_cols)
                input_shape = (1, img_rows, img_cols)
            else:
                X = X.reshape(X.shape[0],img_rows, img_cols, 1)
                input_shape = (img_rows, img_cols, 1)
        else:
            input_shape = X.shape[1:]
            
        self.X = X
        self.input_shape = input_shape
            
                
                

In [4]:
class Machine():
    def __init__(self,X,y,nb_classes=2,fig=True):
        self.nb_classes = nb_classes
        self.set_data(X,y)
        self.set_model()
        self.fig = fig
        
    def set_data(self, X,y):
        nb_classes = self.nb_classes
        self.data = DataSet(X,y,nb_classes)
        print('data.input_shape ', self.data.input_shape)
        
    def set_model(self):
        nb_classes = self.nb_classes
        data = self.data
        self.model = CNN(nb_classes=nb_classes, in_shape=data.input_shape)
    def fit(self, epochs=10, batch_size=128, verbose=1):
        data = self.data
        model = self.model
        
        history = model.fit(data.X_train, data.Y_train, batch_size=batch_size,epochs=epochs,
                           verbose=verbose,validation_data=(data.X_test,data.Y_test))
        
        return history
    
    def run(self, epochs=100, batch_size=128, verbose=1):
        data = self.data
        model = self.model
        fig = self.fig
        
        history = self.fit(epochs=epochs, batch_size=batch_size, verbose=verbose)
        
        score = model.evaluate(data.X_test, data.Y_test, verbose=0)
        
        print('Confusion matrix')
        Y_test_pred = model.predict(data.X_test,verbose=0)
        y_test_pred = np.argmax(Y_test_pred,axis=1)
        print(metrics.confusion_matrix(data.y_test, y_test_pred))
        
        print('Test Score: ',score[0])
        print('Test Accuracy: ', score[1])
        
        suffix = unique_filename('datatime')
        foldname = 'output_'+suffix
        os.makedirs(foldname)
        save_history_history('history_history.npy', history.history, fold=foldname)
        model.save_weights(os.path.join(foldname, 'dl_model.h5'))
        print('Output results are saved in ', foldname)
        
        if fig:
            plt.figure(figsize=(12,4))
            plt.subplot(1,2,1)
            plot_acc(history)
            plt.subplot(1,2,2)
            plot_loss(history)
            plt.show()
        
        self.history=history
        return foldname

In [ ]:
(X,y), (x_test, y_test) = datasets.cifar10.load_data()
m = Machine(X,y,nb_classes=10)
m.run()

(40000, 32, 32, 3) (40000, 1)
X_train shape :  (40000, 32, 32, 3)
40000  train samples
10000  test samples
data.input_shape  (32, 32, 3)
Epoch 1/100
313/313 [==============================] - 2s 6ms/step - loss: 2.3153 - accuracy: 0.1074 - val_loss: 2.2882 - val_accuracy: 0.1355
Epoch 2/100
313/313 [==============================] - 2s 5ms/step - loss: 2.2987 - accuracy: 0.1165 - val_loss: 2.2764 - val_accuracy: 0.1601
Epoch 3/100
313/313 [==============================] - 2s 5ms/step - loss: 2.2858 - accuracy: 0.1303 - val_loss: 2.2660 - val_accuracy: 0.1733
Epoch 4/100
313/313 [==============================] - 2s 5ms/step - loss: 2.2775 - accuracy: 0.1378 - val_loss: 2.2561 - val_accuracy: 0.1972
Epoch 5/100
313/313 [==============================] - 2s 5ms/step - loss: 2.2675 - accuracy: 0.1481 - val_loss: 2.2455 - val_accuracy: 0.2173
Epoch 6/100
313/313 [==============================] - 2s 5ms/step - loss: 2.2575 - accuracy: 0.1613 - val_loss: 2.2344 - val_accuracy: 0.2299
Epoch

313/313 [==============================] - 2s 6ms/step - loss: 1.9312 - accuracy: 0.3087 - val_loss: 1.8587 - val_accuracy: 0.3676
Epoch 58/100
313/313 [==============================] - 2s 6ms/step - loss: 1.9345 - accuracy: 0.3090 - val_loss: 1.8557 - val_accuracy: 0.3694
Epoch 59/100
313/313 [==============================] - 2s 5ms/step - loss: 1.9311 - accuracy: 0.3112 - val_loss: 1.8519 - val_accuracy: 0.3705
Epoch 60/100
313/313 [==============================] - 2s 6ms/step - loss: 1.9274 - accuracy: 0.3115 - val_loss: 1.8483 - val_accuracy: 0.3749
Epoch 61/100
313/313 [==============================] - 2s 5ms/step - loss: 1.9218 - accuracy: 0.3120 - val_loss: 1.8448 - val_accuracy: 0.3719
Epoch 62/100
313/313 [==============================] - 2s 6ms/step - loss: 1.9168 - accuracy: 0.3158 - val_loss: 1.8410 - val_accuracy: 0.3755
Epoch 63/100
313/313 [==============================] - 2s 5ms/step - loss: 1.9126 - accuracy: 0.3165 - val_loss: 1.8366 - val_accuracy: 0.3785
Epoch